# Project 3

*Meghana V Palukuri mvp549*


## Instructions

After completing this Jupyter notebook, please convert it to pdf and submit both the pdf and the original notebook on Canvas **no later than 4:00 pm on May 9, 2019**. The two documents will be graded jointly, so they must be consistent (as in, don't change the Jupyter notebook without also updating the converted pdf!).

All results presented **must** have corresponding code. Any answers/results given without the corresponding python code that generated the result will be considered absent. All code reported in your final project document should work properly.

Before submitting the Jupyter notebook part, please re-run all cells by clicking "Kernel" and selecting "Restart & Run All."

The project consists of two problems. For both problems, please follow these guidelines:

- Final output needs to be nicely formatted and human readable. For example, if your result is a count, don't just print the value of the count, print "The count is: ...".
- For each problem, limit your total code to less than 100 lines.
- Write comments and explanatory text, so we understand what you are doing.
- Do not print out large datasets, such as an entire genome, or a list of all genes in a genome, etc.
- Verify that nothing of importance (code, comments, other text) is cut off in your final pdf. 


## Problem 1

The bacteria called *Salmonella enterica* Typhimurium are pathogenic bacteria closely related to *E. coli*. They cause typhoid fever in humans. There are many different *S. enterica* Typhimurium strains, and here we will compare two such strains, LT2 and CT18. LT2 is the canonical strain that is most commonly used as a reference. CT18 is another widely used reference.

Before we can work with these two genomes, we need to download them. Note: Running the next cell may take a few minutes.

In [1]:
from Bio import Entrez
Entrez.email = "meghana@ices.utexas.edu" # put your email here

# Download S. enterica strain LT2 and write into file "S_enterica_LT2.gb":
download_handle = Entrez.efetch(db="nucleotide", id="NC_003197", rettype="gbwithparts", retmode="text")
out_handle = open("S_enterica_LT2.gb", "w")
out_handle.write(download_handle.read())
download_handle.close()
out_handle.close()
print("Downloaded S. enterica LT2")

# Download S. enterica strain CT18 and write into file "S_enterica_CT18.gb":
download_handle = Entrez.efetch(db="nucleotide", id="NC_003198", rettype="gbwithparts", retmode="text")
out_handle = open("S_enterica_CT18.gb", "w")
out_handle.write(download_handle.read())
download_handle.close()
out_handle.close()
print("Downloaded S. enterica CT18")

Downloaded S. enterica LT2
Downloaded S. enterica CT18


**Problem 1a (30 pts):** How many named protein-coding genes are in *S. enterica* LT2? And how many of these genes have synonyms in *S. enterica* CT18?

Hint: Gene names have been defined for the LT2 strain. You can find these names in the "gene" qualifier of CDS features. When equivalent genes exist in CT18, they are listed under the "gene_synonym" qualifer of the CDS features. As an example, manually open the two genome files and look for the "thrL" gene in each genome.

First I will make a list of the protein-coding genes in *S. enterica* LT2 - the length of this list answers the first part of the question. I will also make a list of the synonyms for each of the genes in *S. enterica* CT18. Then for each of the protein-coding genes in *S. enterica* LT2, I will search for a match in the list of gene synonyms of the genes in *S. enterica* CT18. If there is a match, the gene is appended to a list, genes_with_synonyms - the length of this list gives the answer to the 2nd part of the question. If there is no match, the gene is appended to another list, genes_without_synonyms for use in the next question. 

In [15]:
from Bio import SeqIO

# read in the LT2 genome
in_handle = open("S_enterica_LT2.gb", "r")
record_LT2 = SeqIO.read(in_handle, "genbank")
in_handle.close()

# read in the CT18 genome
in_handle = open("S_enterica_CT18.gb", "r")
record_CT18 = SeqIO.read(in_handle, "genbank")
in_handle.close()

# initialize gene list for LT2 to empty list
gene_list_LT2 = []
for rec_feat in record_LT2.features: # Loop through features
    if rec_feat.type == "CDS": # Check if coding sequence
        if 'gene' in rec_feat.qualifiers: # To prevent key errors in case a gene key is not present
            gene_list_LT2.append(rec_feat.qualifiers['gene'][0]) # Add gene name to the list 
            
print("No. of protein coding genes in S. enterica LT2 = ",\
      len(gene_list_LT2)) # Finding length of gene list

No. of protein coding genes in S. enterica LT2 =  3242


In [3]:
gene_synonym_list_CT18 = [] # initialize gene synonym list for CT18 to empty list
for rec_feat in record_CT18.features:
    if rec_feat.type == "CDS":
        if 'gene_synonym' in rec_feat.qualifiers: # To prevent key errors in case a gene key is not present
            gene_synonym_list_CT18.append(rec_feat.qualifiers['gene_synonym'][0]) # Add gene synonym name to the list

In [14]:
genes_with_synonyms = [] # list of LT2 genes which have synonyms in CT18
genes_without_synonyms = [] # list of LT2 genes which do NOT have synonyms in CT18
for gene in gene_list_LT2: # Looping through gene list in LT2
    if gene in gene_synonym_list_CT18: # If it has a synonym in CT18 add it to the first list
        genes_with_synonyms.append(gene)
    else:
        genes_without_synonyms.append(gene) # Otherwise, add it to the other list. 
    
# Finding length of genes_with_synonyms    
print("No. of protein coding genes in S. enterica LT2 that have synonyms in S. enterica CT18 = ",\
      len(genes_with_synonyms))

No. of protein coding genes in S. enterica LT2 that have synonyms in S. enterica CT18 =  1515


To make a list of the protein-coding genes in *S. enterica* LT2, I loop through all the features and if the feature type is a coding DNA sequence, I then check if there is a gene name defined by seeing if the 'gene' qualifier exists. The length of this list, i.e the number of protein-coding genes in *S. enterica* LT2 is 3242. To make a list of the synonyms for each of the genes in *S. enterica* CT18, I am checking again for CDS features and now checking if the 'gene synonym' qualifier exists. To answer the 2nd question, I loop through the list of genes in *S. enterica* LT2 and if there is a match in the list of gene synonyms from *S. enterica* CT18, I add it to the final solution list genes_with_synonyms. The length of genes_with_synonyms is 1515, which is the number of genes in LT2 that have synonyms in CT18. 

**Problem 1b (20 pts):** How many of the named genes in LT2 without a synonym in CT18 have their product listed as "hypothetical protein"?

As before, we first loop through the features and for CDS type of features which have the gene qualifier defined, we check if this gene does not have a synonym in CT18 by checking if it is present in the list genes_without_synonyms computed in the previous question. Next, we check if the product qualifier is called hypothetical protein, in which case we advance the counter (initially initialized to 0) by 1. The final counter value gives the answer to this question.

In [16]:
hyp_prots = 0
for rec_feat in record_LT2.features:
    if rec_feat.type == "CDS":
        if 'gene' in rec_feat.qualifiers: # To prevent key errors in case a gene key is not present
            if rec_feat.qualifiers['gene'][0] in genes_without_synonyms:
                if rec_feat.qualifiers['product'][0] == "hypothetical protein":
                    hyp_prots += 1

print("No. of protein coding genes in LT2 without a synonym in CT18 with their product listed as hypothetical protein = "\
      ,hyp_prots)

No. of protein coding genes in LT2 without a synonym in CT18 with their product listed as hypothetical protein =  244


The list genes_without_synonyms contains genes in LT2 which do not have synonyms in CT18. We look at genes in this list and count the number of hypothetical proteins, which was found to be 244.

## Problem 2

**(50 pts)**

Ask a question about the genomes from Problem 1 and then write python code that generates an answer. The question does not have to be conceptual, and it can be about only one of the two genomes or about the two genomes jointly.

For full credit, the answer code must meet the following conditions:

- contains at least one `for` loop
- contains at least one `if` statement
- uses at least one list or dictionary
- uses at least one regular expression

**Question:** *How many proteins related to pathogenecity islands are there in S. enterica LT2 and S. enterica CT18?*

Since the strains are harmful, I thought it would be a good idea to find genes related to pathogenic activity. When I was looking through the genes in LT2, I found that some of them had Pathogenecity island given in the notes section and was curious to find how many such genes were present. 

For this, I look at coding DNA sequences, and see if their 'note' qualifier has a mention of the phrase 'Pathogenicity Island' using regular expression matching after converting the note to lower case. If there is a match, I add the gene to a list. Since all the matches may not have corresponding gene names defined, I maintain a count of those that do not have names and add this count to the length of the list to get the final count of the number of genes related to pathogenecity islands.

In [6]:
import re
def get_pathogen_count(record_name): # Function to get number of pathogenecity islands given a genbank record.  
    pathogen_gene_list = [] # List for genes related to pathogenecity islands
    ct_no_gn_name = 0 # Count of those related to pathogenecity islands without defined gene names. 
    for rec_feat in record_name.features:
        if rec_feat.type == "CDS": # Coding DNA sequence check
            if 'note' in rec_feat.qualifiers: # To prevent key error if note is not present 
                # Convert note to lower case and regex compare with pathogenecity island.
                match = re.search(r"pathogenicity island",rec_feat.qualifiers['note'][0].lower())
                if match:
                    if 'gene' in rec_feat.qualifiers: # To prevent key errors in case a gene key is not present
                        pathogen_gene_list.append(rec_feat.qualifiers['gene'][0])
                    else:
                        ct_no_gn_name += 1
    return (len(pathogen_gene_list)+ct_no_gn_name,pathogen_gene_list) # Return both count and gene list 

In [7]:
(ct_LT2,pathogen_gene_list_LT2) = get_pathogen_count(record_LT2)
print("No. of proteins related to pathogenecity islands in S. enterica LT2 = ",ct_LT2)
print("Genes encoding these proteins (if known):",pathogen_gene_list_LT2)

No. of proteins related to pathogenecity islands in S. enterica LT2 =  15
Genes encoding these proteins (if known): ['copS', 'copR', 'sugR', 'rhuM', 'rmbA', 'misL', 'fidL', 'marT', 'slsA', 'cigR', 'mgtB', 'mgtC']


In [8]:
(ct_CT18,pathogen_gene_list_CT18) = get_pathogen_count(record_CT18)
print("No. of proteins related to pathogenecity islands in S. enterica CT18 = ",ct_CT18)

No. of proteins related to pathogenecity islands in S. enterica CT18 =  0


We can see that *S. enterica* LT2 has 15 genes related to Pathogenecity Islands. Of these, 12 have defined gene names and are printed above. *S. enterica* CT18 on the other hand has no genes with notes containing the phrase - pathogenecity island. This could be because the strain was not annotated completely. Since *S. enterica* LT2 has pathogenecity island annotations in its notes, we can perhaps check for genes in *S.enterica* L2 which have gene synonyms in *S. enterica* CT18 that have notes saying pathogenicity island, to determine genes in *S. enterica* CT18 related to pathogenecity islands. 

In [9]:
def get_pathogen_count_in_genes(record_name,gene_list): 
    # Function inputs: genbank record,list of genes of interest to check for pathogenicity islands
    ct = 0 # counter
    for rec_feat in record_name.features:
        if rec_feat.type == "CDS": 
            # Check to prevent key errors in case a gene key is not present,
            # since we are looking at genes only with defined names here            
            if 'gene' in rec_feat.qualifiers: 
                # Checking if the gene is present in the list of genes of interest           
                if rec_feat.qualifiers['gene'][0] in gene_list:                 
                    if 'note' in rec_feat.qualifiers: # Check if note is present
                        # Check for match with pathogenecity island
                        match = re.search(r"pathogenicity island",rec_feat.qualifiers['note'][0].lower())
                        if match:
                            ct += 1
    return ct

In [10]:
ct_CT18 = get_pathogen_count_in_genes(record_LT2,genes_with_synonyms)
print("No. of proteins related to pathogenecity islands in S. enterica CT18 = ",ct_CT18)

No. of proteins related to pathogenecity islands in S. enterica CT18 =  0


Unfortunately, this method also does not yield any genes in *S. enterica* CT18 which may be related to pathogenecity islands. This could mean that *S. enterica* CT18 does not actually have pathogenecity islands. But more likely is that its annotations do not note the presence of pathogenecity islands and we will have to check gene synonyms of a well annotated organism (better than *S. enterica* LT2!) to find the answer. Also, here we are checking only the note qualifier - it may be possible that pathogenecity information is given in or can be derived from a different qualifier. 